In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [3]:
# Hyperparameters
sequence_length = 28
input_size = 28 # The number of expected features in the input x
num_layers = 2 # Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final results. Default: 1
hidden_size = 128 #T he number of features in the hidden state h
num_epochs = 5
num_classes = 10
learning_rate = 0.01
batch_size = 100

In [4]:
train_data = torchvision.datasets.MNIST(root = "../../data/",
                                       train=True,
                                    download=True,
                                       transform=transforms.ToTensor())
test_data = torchvision.datasets.MNIST(root = "../../data/",
                                       train=False,
                                       transform=transforms.ToTensor())

Processing...
Done!


In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                          shuffle=True,
                                          batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                               shuffle=True,
                                               batch_size=batch_size)

In [6]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True) 
        # batch_first – If True, then the input and output tensors are provided as (batch, seq, feature). Default: False
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) # (num_layers, batch, hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) # (num_layers, batch, hidden_size)
            
        # Forward propagate LSTM
        out, (hn, cn) = self.lstm(x, (h0, c0)) # out: tensor of shape (seq_len, batch, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        
        return(out)           

In [7]:
model = RNN(input_size, hidden_size, num_layers, num_classes) 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
 for i, (imgs, labels) in enumerate(train_loader):
    imgs = imgs.reshape(-1, sequence_length, input_size).to(device)
    labels = labels.to(device)
    print("Input size: " + str(imgs.size()))
    outputs = model(imgs)
    print("Output size: " + str(outputs.size()))
    x, pred = torch.max(outputs.data, 1)
    print(x.size())
    print(pred[0])
    
    break

Input size: torch.Size([100, 28, 28])
Output size: torch.Size([100, 10])
torch.Size([100])
tensor(9)


In [31]:
# Another Example

rnn = nn.LSTM(28, 128, 2) # (input_size, hidden_size, num_layers)
input = torch.randn(28, 100, 28) # (seq_len, batch, input_size)
h0 = torch.randn(2, 100, 128) # (num_layers, batch, hidden_size)
c0 = torch.randn(2, 100, 128) # (num_layers, batch, hidden_size)
output, (hn, cn) = rnn(input, (h0, c0)) 
print(output.size()) # (seq_len, batch, hidden_size)
print(hn.size())     # (num_layers, batch, hidden_size)
print(output[:, -1, :].size())

torch.Size([28, 100, 128])
torch.Size([2, 100, 128])
torch.Size([28, 128])


In [52]:
# Train the model
total_step = len(train_loader) # Number of batches
for epoch in range(num_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs = imgs.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        pred = model(imgs)
        loss = criterion(pred, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.6404
Epoch [1/5], Step [200/600], Loss: 0.1939
Epoch [1/5], Step [300/600], Loss: 0.2997
Epoch [1/5], Step [400/600], Loss: 0.0978
Epoch [1/5], Step [500/600], Loss: 0.0570
Epoch [1/5], Step [600/600], Loss: 0.0698
Epoch [2/5], Step [100/600], Loss: 0.1266
Epoch [2/5], Step [200/600], Loss: 0.0626
Epoch [2/5], Step [300/600], Loss: 0.0210
Epoch [2/5], Step [400/600], Loss: 0.1645
Epoch [2/5], Step [500/600], Loss: 0.0315
Epoch [2/5], Step [600/600], Loss: 0.0647
Epoch [3/5], Step [100/600], Loss: 0.0985
Epoch [3/5], Step [200/600], Loss: 0.1182
Epoch [3/5], Step [300/600], Loss: 0.0924
Epoch [3/5], Step [400/600], Loss: 0.0480
Epoch [3/5], Step [500/600], Loss: 0.1606
Epoch [3/5], Step [600/600], Loss: 0.0337
Epoch [4/5], Step [100/600], Loss: 0.0194
Epoch [4/5], Step [200/600], Loss: 0.1031
Epoch [4/5], Step [300/600], Loss: 0.0674
Epoch [4/5], Step [400/600], Loss: 0.1979
Epoch [4/5], Step [500/600], Loss: 0.0215
Epoch [4/5], Step [600/600], Loss:

In [56]:
# Test model
with torch.no_grad():
    total = correct = 0
    
    for i, (imgs, labels) in enumerate(test_loader):
        imgs = imgs.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        outputs = model(imgs)
        _, pred = torch.max(outputs.data, 1) # Returns (Value, index)
        
        total += labels.size(0)
        correct += (pred == labels).sum(0)
        
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

Test Accuracy of the model on the 10000 test images: 98 %


In [ ]:
torch.save(model.state_dict(), "RNN.ckpt")